#Ejercicio 9: Uso de la API de Google Gemini
En este ejercicio vamos a aprender a utilizar la API de OpenAI
## 1. Uso básico
El siguiente código sirve para conectarse con la API de Google Gemini de forma básica

In [1]:
!pip install -q -U google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.


In [6]:
from dotenv import load_dotenv

load_dotenv()

import os

api_key = os.getenv('Gemini_API_KEY')



In [20]:
from google import genai
from google.colab import userdata

# 1. Obtener la clave de forma privada desde los Secrets
api_key = userdata.get('GEMINI_API_KEY')

# 2. Configurar el cliente usando la variable
client = genai.Client(api_key=api_key)

# 3. Prueba rápida de conexión
try:
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents="Conexión exitosa, responde 'OK'"
    )
    print(f"Estado de la API: {response.text}")
except Exception as e:
    print(f"Error de conexión: {e}")

Estado de la API: OK


In [7]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents="Explain how AI works in a few words",
)

print(response.text)

AI processes vast amounts of data to recognize patterns and make predictions.


## 2. Retrieval
### 2.1 Cargo el corpus de 20 News Groups


In [8]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data

In [9]:
import pandas as pd
df = pd.DataFrame(newsgroupsdocs, columns=["text"])
df


,text
0,\n\nI am sure some bashers of Pens fans are pr...
1,My brother is in the market for a high-perform...
2,\n\n\n\n\tFinally you said what you dream abou...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...
4,1) I have an old Jasmine drive which I cann...
...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...
18842,\nNot in isolated ground recepticles (usually ...
18843,I just installed a DX2-66 CPU in a clone mothe...
18844,\nWouldn't this require a hyper-sphere. In 3-...


### 2.2 Transformo a embeddings

In [10]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,text,text_norm
0,\n\nI am sure some bashers of Pens fans are pr...,I am sure some bashers of Pens fans are pretty...
1,My brother is in the market for a high-perform...,My brother is in the market for a high-perform...
2,\n\n\n\n\tFinally you said what you dream abou...,Finally you said what you dream about. Mediter...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,Think! It's the SCSI card doing the DMA transf...
4,1) I have an old Jasmine drive which I cann...,1) I have an old Jasmine drive which I cannot ...


In [11]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  I am sure some bashers of Pens fans are pretty...
 1       1         0  My brother is in the market for a high-perform...
 2       2         0  Finally you said what you dream about. Mediter...
 3       2         1  urds and Turks once upon a time! Ohhhh so swed...
 4       3         0  Think! It's the SCSI card doing the DMA transf...,
 38871)

In [12]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [13]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

Batches:   0%|          | 0/2430 [00:00<?, ?it/s]

In [14]:
print(embeddings.shape, embeddings.dtype)

(38871, 768) float32


### 2.3 Creo una query y hago la búsqueda


In [15]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

In [16]:
import numpy as np

def manual_search(query_embedding, embeddings_matrix, k=5):
    """
    Calcula la similitud entre la query y todos los embeddings
    sin usar bases de datos externas.
    """
    # 1. Asegurar que los vectores tengan la forma correcta
    # query_embedding suele ser (1, 768) y embeddings_matrix (N, 768)
    query_vec = query_embedding.flatten()

    # 2. Calcular el Producto Punto (Similitud)
    # Al estar normalizados, esto equivale a la Similitud Coseno
    # Resultado: un array de scores de tamaño N
    scores = np.dot(embeddings_matrix, query_vec)

    # 3. Obtener los índices de los 'k' valores más altos
    # argsort ordena de menor a mayor, por eso usamos [::-1] para invertirlo
    top_k_indices = np.argsort(scores)[::-1][:k]

    # 4. Formatear los resultados cruzando con el DataFrame original
    results = []
    for idx in top_k_indices:
        results.append({
            "id": idx,
            "score": float(scores[idx]),
            "text": chunks_df.iloc[idx]["text"],
            "metadata": {
                "doc_id": int(chunks_df.iloc[idx]["doc_id"]),
                "chunk_id": int(chunks_df.iloc[idx]["chunk_id"])
            }
        })

    return results


Obtengo los 5 documentos más similares a mi query

In [17]:

# --- Ejemplo de ejecución ---
query_text = "how much lead is in a typical battery?"
query_vec = embed_query(query_text) # Función de la Parte 1

# Realizamos la búsqueda manual sobre la matriz 'embeddings'
resultados_manuales = manual_search(query_vec, embeddings, k=5)

print(f"Resultados Manuales (NumPy) para: '{query_text}'\n" + "="*70)
for r in resultados_manuales:
    print(f"Score: {r['score']:.4f} | Texto: {r['text'][:90]}...")

Resultados Manuales (NumPy) para: 'how much lead is in a typical battery?'
Score: 0.8514 | Texto: The lead-acid secondary cell releases energy (electricity) with the following chemical rea...
Score: 0.8346 | Texto: are dissolved). -The level of H30+ ions in the acid solution decreases (i.e. the solution ...
Score: 0.8320 | Texto: on in a newer battery) a dead short results. I have seen products in automotive shops to c...
Score: 0.8306 | Texto: I am not a battery expert, but from recent reading, a lead acid battery will self discharg...
Score: 0.8275 | Texto: ing something like 200 pounds of eq. Well, a BATSE detector needs lead shielding to protec...


In [22]:
from google import genai
from google.colab import userdata

# 1. Configuración segura del cliente
# Extraemos la clave del almacén de secretos de Colab
api_key = userdata.get('GEMINI_API_KEY')
client = genai.Client(api_key=api_key)

# 2. Preparar el contexto combinando los resultados de la búsqueda manual
# Esto toma los fragmentos de texto encontrados por NumPy
contexto_textos = "\n".join([f"- {r['text']}" for r in resultados_manuales])

# 3. Diseñar el Prompt robusto (RAG)
prompt = f"""
Eres un asistente experto. Basándote exclusivamente en la información proporcionada en el "Contexto" a continuación,
responde a la siguiente pregunta del usuario de forma resumida y clara.
Si la respuesta no está en el contexto, indícalo.

Pregunta: {query_text}

Contexto:
{contexto_textos}

Respuesta:
"""

# 4. Generar la respuesta usando el método del cliente
# He puesto "gemini-2.0-flash-exp" porque es el modelo más actual que acepta la SDK,
# pero puedes cambiarlo a "gemini-1.5-flash" si lo prefieres.
try:
    response = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=prompt
    )

    # 5. Mostrar el resultado final
    print(f"--- RESUMEN DE INTELIGENCIA ARTIFICIAL (GEMINI) ---")
    print(response.text)

except Exception as e:
    print(f"Error al generar contenido: {e}")

--- RESUMEN DE INTELIGENCIA ARTIFICIAL (GEMINI) ---
La información proporcionada en el contexto no indica la cantidad de plomo que contiene una batería típica; solo describe la reacción química, el proceso de descarga y el mantenimiento de las celdas de plomo-ácido.
